In [1]:
import numpy as np
import matplotlib.pyplot as plt
import imageio

class LassoRegression:
    def __init__(self, learning_rate=0.01, n_epochs=100, l1_lambda=0.1):
        self.lr = learning_rate
        self.n_epochs = n_epochs
        self.l1_lambda = l1_lambda
        self.weights = None
        self.bias = None
        self.loss_history = []
        self.frames = []  # Store frames for animation

    def _soft_threshold(self, w, lambda_val):
        """Soft-thresholding function for L1 regularization"""
        return np.sign(w) * np.maximum(np.abs(w) - lambda_val, 0)

    def fit(self, X, y, save_path="Lasso_Regression.gif"):
        n_samples, n_features = X.shape
        self.weights = np.random.randn(n_features)  # Initialize weights
        self.bias = 0
        
        for epoch in range(self.n_epochs):
            y_pred = np.dot(X, self.weights) + self.bias
            error = y_pred - y
            
            # Compute gradients
            dw = (1 / n_samples) * np.dot(X.T, error)
            db = (1 / n_samples) * np.sum(error)
            
            # Apply L1 soft-thresholding
            self.weights -= self.lr * dw
            self.weights = self._soft_threshold(self.weights, self.lr * self.l1_lambda)
            self.bias -= self.lr * db

            # Compute loss (MSE + L1 penalty)
            mse_loss = np.mean(error ** 2)
            l1_penalty = self.l1_lambda * np.sum(np.abs(self.weights))
            loss = mse_loss + l1_penalty
            self.loss_history.append(loss)

            # Save plot for GIF
            self._save_plot(X, y, epoch)
        
        # Save GIF animation
        imageio.mimsave(save_path, self.frames, duration=0.1)
        print(f"GIF saved successfully as {save_path}")

    def _save_plot(self, X, y, epoch):
        """Visualizes weight shrinking over epochs."""
        fig, ax = plt.subplots(1, 2, figsize=(12, 5))

        # Scatter plot of data points
        ax[0].scatter(X[:, 0], y, color='blue', alpha=0.5, label="Data points")
        x_line = np.linspace(X[:, 0].min(), X[:, 0].max(), 100)
        y_line = self.weights[0] * x_line + self.bias
        ax[0].plot(x_line, y_line, color='red', linewidth=2, label="Lasso Fit")
        ax[0].set_title(f"Regression Fit (Epoch {epoch+1})")
        ax[0].set_xlabel("Feature")
        ax[0].set_ylabel("Target")
        ax[0].legend()

        # Weight shrinking visualization
        ax[1].bar(range(len(self.weights)), self.weights, color='green')
        ax[1].set_title(f"Weights Shrinking (Epoch {epoch+1})")
        ax[1].set_xlabel("Feature Index")
        ax[1].set_ylabel("Weight Value")
        ax[1].grid(True, linestyle="--", alpha=0.3)

        # Save frame for GIF
        fig.canvas.draw()
        image = np.array(fig.canvas.renderer.buffer_rgba())
        self.frames.append(image)
        plt.close(fig)

# Generate synthetic data
np.random.seed(42)
X = np.random.randn(100, 1)  # Single feature
y = 3 * X[:, 0] + np.random.randn(100) * 0.5  # Linear relation with noise

# Train Lasso Regression model and save GIF
model = LassoRegression(learning_rate=0.1, n_epochs=50, l1_lambda=0.5)
model.fit(X, y, save_path="Lasso_Regression.gif")

# Final output
print("\n=== Final Model ===")
print(f"Weights: {model.weights}")
print(f"Bias: {model.bias}")
print(f"Final Loss: {model.loss_history[-1]:.4f}")


GIF saved successfully as Lasso_Regression.gif

=== Final Model ===
Weights: [2.28705642]
Bias: -0.06961704970116642
Final Loss: 1.7029
